**This notebook contains a script that combines all the SETA POE documents into one submission Folder**

**Add Local Library to Path**

In [51]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [52]:
import pandas as pd
from local_library import list_files
from local_library import create_folder
from local_library import import_worksheet
from local_library import export_worksheet

**Temp Library**

In [53]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

#authenticate Google
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)


def move_file(old_folder_id, new_folder_id, exclusion_array=[]):
    """
    Moves a google drive files from one folder to another.
    
    Args:
        old_folder_id: A string representing the old folder google drive id.
        new_folder_id: A string representing the new folder google drive id.
        exclusion_array: An array of strings containing the titles to be excluded from the copy.
        
    Returns:
        _: A boolean representing success or failure
    """
    
    files = list_files(old_folder_id)
    for file in files:
        if file['title'] not in exclusion_array:
            copy_file(file['id'], file['title'], new_folder_id)
    return True
    
def copy_file(file_id, file_title, new_folder_id):
    """
    Creates copy of file in destination folder
    
    Args:
        file_id: I.D of the file to be copied
        file_title: Name of the file to be copied
        new_folder_id: Folder where copy will be stored.
    Return:
        none
    """
    
    copied_file = {"title": file_title,
                    'parents': [{
            'kind': 'drive#fileLink',
            'id': new_folder_id
        }]}

    drive.auth.service.files().copy(fileId=file_id,
                                    body=copied_file,
                                    supportsAllDrives=True
    ).execute()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Map Documents folders**

In [54]:
files = list_files("1W04tIGaWpqSgKsaP6xSjrUtP1OsVcaPX")

In [55]:
student_documents_folders = []
for file in files:
    student_documents_folders.append([file['title'], file['id']])


In [56]:
documents_df = pd.DataFrame(student_documents_folders, columns=["Name", "Documents ID"])

**Map Evidence Folders**

In [57]:
files_2 = list_files("1-2tkFz7qrd2MD-vvudm0AlyK46YFt5so")

In [58]:
student_evidence_folders = []
for file in files_2:
    student_evidence_folders.append([file['title'], file['id']])

In [59]:
evidence_df = pd.DataFrame(student_evidence_folders, columns=["Name", "Evidence ID"])

**Merge Documents and Evidence Folders**

In [60]:
folders_df = pd.merge(documents_df, evidence_df, how="inner")

In [61]:
folders_df = folders_df[folders_df['Name'] == 'Adam Saul David Becker']
folders_df.head()

,Name,Documents ID,Evidence ID
51,Adam Saul David Becker,19oj4tcbwqgRsGEAia15LRAyVKROwUE_F,16dCBkcvMqP5xURfMNBSRZIb98jIyPNrP


**Create Local Library**

In [67]:
def create_poe_folder(poe_parent_folder_id, poe_folder_title):
    """
    Creates the student POE folder.
    
    Args:
        poe_parent_folder_id: A string representing the id of the poe parent folder
        poe_folder_title: A string representing the students fullname.
    """
    
    #declare global variables
    global poe_folder
    
    poe_folder = create_folder(poe_parent_folder_id, poe_folder_title)
    
    
def create_documents_folder(poe_documents_folder_id, evidence_documents_folder_id):
    """
    Creates document folder and copies documents files to folder.
    
    Args:
        poe_documents_folder_id: A string representing the id of the old poe documents folder.
        evidence_documents_folder_id A string representing the id of the old evidence folder.
    """
    
    #declare global variables
    global documents_folder, evidence_folders
    
    #create documents folder
    documents_folder = create_folder(poe_folder, 'POE Documents')
    
    #move all poe documents into documents folder
    move_file(poe_documents_folder_id, documents_folder)
    
    #move all loose documents from evidence folder to documents folder
    exclusion_array = ['documents', 'exams', 'Post Assessment Documents', 'technical reports', 'video demonstrations', 'video presentations']
    move_file(evidence_documents_folder_id, documents_folder, exclusion_array)
    
    #move all personal documents into documents folder
    evidence_folders = list_files(evidence_documents_folder_id)
    for folder in evidence_folders:
        if folder['title'] == 'documents':
            move_file(folder['id'], documents_folder)
            
            
def create_presentation_folder():
    """
    Creates presentation folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global presentation_folder, presentation_instruments_folder, presentation_evidence_folder
    
    #create presentation folder
    presentation_folder = create_folder(poe_folder, 'Module 1 - Presentation')
    
    #add presentation instruments folder
    presentation_instruments_folder = create_folder(presentation_folder, 'instruments')
    
    #add presentation evidence folder
    presentation_evidence_folder = create_folder(presentation_folder, 'evidence')
    
    #move all presentation evidence into presentation evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'video presentations':
            move_file(folder['id'], presentation_evidence_folder)
            
    #move all presentation post assessments docs into presentation folder
    exclusion_array = ['demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], presentation_folder,exclusion_array)
            
            
def create_demonstration_folder():
    """
    Creates demonstration folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global demonstration_folder, demonstration_instruments_folder, demonstration_evidence_folder
    
    #create demonstration folder
    demonstration_folder = create_folder(poe_folder, 'Module 2 - Demonstration')
    
    #add demonstration instruments folder
    demonstration_instruments_folder = create_folder(demonstration_folder, 'instruments')
    
    #add demonstration evidence folder
    demonstration_evidence_folder = create_folder(demonstration_folder, 'evidence')
    
    #move all demonstration evidence into demonstration evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'video demonstrations':
            move_file(folder['id'], demonstration_evidence_folder)
            
    #move all demonstration post assessments docs into demonstration folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], demonstration_folder,exclusion_array)
            
            
def create_reports_folder():
    """
    Creates reports folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global reports_folder, reports_instruments_folder, reports_evidence_folder
    
    #create reports folder
    reports_folder = create_folder(poe_folder, 'Module 3 - Reports')
    
    #add reports instruments folder
    reports_instruments_folder = create_folder(reports_folder, 'instruments')
    
    #add reports evidence folder
    reports_evidence_folder = create_folder(reports_folder, 'evidence')
    
    #move all report evidence into report evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'technical reports':
            move_file(folder['id'], reports_evidence_folder)
    
    #move all reports post assessments docs into reports folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], reports_folder,exclusion_array)
            
            
def create_exams_folder():
    """
    Creates exam folder and subfolders and copies files into them.
    """
    
    #declare global variables
    global exams_folder, exams_formatives_folder, exams_summatives_folder
    
    #create exams folder
    exams_folder = create_folder(poe_folder, 'Module 4 - Exams')
    
    #add exams formative folder
    exams_formatives_folder = create_folder(exams_folder, 'formatives')
    
    #add exams summative folder
    exams_summatives_folder = create_folder(exams_folder, 'summatives')
    
    #move all exam evidence into exam evidence folder
    for folder in evidence_folders:
        if folder['title'] == 'exams':
            exams_folders = list_files(folder['id'])
            for exam_folder in exams_folders:
                if exam_folder['title'] == 'formative':
                    move_file(exam_folder['id'], exams_formatives_folder)
                if exam_folder['title'] == 'summative':
                    move_file(exam_folder['id'], exams_summatives_folder)
                    
    #move all exams post assessments docs into exams folder
    exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','report_feedback.pdf','report_review.pdf']
    for folder in evidence_folders:
        if folder['title'] == 'Post Assessment Documents':
            move_file(folder['id'], exams_folder,exclusion_array)
    

**Main Function**

In [ ]:
array_df = [['poe_folder_title', 'poe_folder', 'documents_folder',
                'presentation_folder', 'presentation_instruments_folder', 'presentation_evidence_folder',
                'demonstration_folder', 'demonstration_instruments_folder', 'demonstration_evidence_folder',
                'reports_folder', 'reports_instruments_folder', 'reports_evidence_folder',
                'exams_folder', 'exams_formatives_folder', 'exams_summatives_folder']]

for index, row in folders_df.itterows():
    
    #declare global variables
    poe_parent_folder_id = row['parent_folder_id']
    poe_folder_title = row['full_name']
    poe_documents_folder_id = row['documents_id']
    evidence_documents_folder_id = row['evidence_id']
    
    #declare global variables
    poe_folder = ''
    documents_folder = ''
    evidence_folders = []
    presentation_folder = ''
    presentation_instruments_folder = ''
    presentation_evidence_folder = ''
    demonstration_folder = ''
    demonstration_instruments_folder = ''
    demonstration_evidence_folder = ''
    reports_folder = ''
    reports_instruments_folder = ''
    reports_evidence_folder = ''
    exams_folder = ''
    exams_formatives_folder = ''
    exams_summatives_folder = ''
    
    #execute main functions
    create_poe_folder(poe_parent_folder_id, poe_folder_title)
    create_documents_folder(poe_documents_folder_id, evidence_documents_folder_id)
    create_presentation_folder()
    create_demonstration_folder()
    create_reports_folder()
    create_exams_folder()
    
    #create new array
    array_df.append([poe_folder_title, poe_folder, documents_folder,
                presentation_folder, presentation_instruments_folder, presentation_evidence_folder,
                demonstration_folder, demonstration_instruments_folder, demonstration_evidence_folder,
                reports_folder, reports_instruments_folder, reports_evidence_folder,
                exams_folder, exams_formatives_folder, exams_summatives_folder])

**Declare global variables**

In [62]:
poe_parent_folder_id = '12vNvTMr7wOYYrr6pG_juR13mNX_K7Q8B'
poe_folder_title = 'Adam Saul David Becker'
poe_documents_folder_id = '19oj4tcbwqgRsGEAia15LRAyVKROwUE_F'
evidence_documents_folder_id = '16dCBkcvMqP5xURfMNBSRZIb98jIyPNrP'
poe_folder = ''
documents_folder = ''
evidence_folders = []
presentation_folder = ''
presentation_instruments_folder = ''
presentation_evidence_folder = ''
demonstration_folder = ''
demonstration_instruments_folder = ''
demonstration_evidence_folder = ''
reports_folder = ''
reports_instruments_folder = ''
reports_evidence_folder = ''
exams_folder = ''
exams_formatives_folder = ''
exams_summatives_folder = ''

**Run Main Function**

In [64]:
create_poe_folder(poe_parent_folder_id, poe_folder_title)
create_documents_folder(poe_documents_folder_id, evidence_documents_folder_id)
create_presentation_folder()
create_demonstration_folder()
create_reports_folder()
create_exams_folder()

**Print Variables Arrays**

In [65]:
output_array = [poe_folder_title, poe_folder, documents_folder,
                presentation_folder, presentation_instruments_folder, presentation_evidence_folder,
                demonstration_folder, demonstration_instruments_folder, demonstration_evidence_folder,
                reports_folder, reports_instruments_folder, reports_evidence_folder,
                exams_folder, exams_formatives_folder, exams_summatives_folder]

In [66]:
print(output_array)

['Adam Saul David Becker', '1g0Z5b-HQiND-6an3JoAatm3RG1q--Gym', '1Cz6q3JWfNVLRWNbhbJMhC6ub1jRyd9Ex', '1A900Wdf8C0KbC3ycBwJnhFdcg2qkb9q8', '1-OOxk7hW150RIPBXMbUQlE5iSPax1rbK', '1g0cQtA64vcxtoBFhq9mQkZ5eWton2VLR', '1Vejk80AL1KacVMnvJUHGHXtZc8pXttBN', '18AICAv7zrMdr6FA_Mv8pbkoXGFGwCgwy', '1OsN4JHRYYOB2I8WDccQNz6VTnilYtdks', '1H1AjohP2MwqSgyDS0sScSL-ZrwBEAve_', '1CQwx1nj5omBix9PiHve9NpiKQOcN4TyY', '127lAQG-ArZRf-u6cfJm69y_22xXFscJ2', '1nq77MYsRRbDH5n91IPMUBQ2MuVAsf9p7', '1ngIGCdf5BA_ECT1zeqKGVXJZqArDt5JZ', '1RR8JC9QSduK8iG8h2wGgOW1HwLTV3O3T']


**Create Parent Folder**

In [13]:
poe_folder = create_folder('12vNvTMr7wOYYrr6pG_juR13mNX_K7Q8B', 'Adam Saul David Becker')

**Documents folder**

In [14]:
#create documents folder
documents_folder= create_folder(poe_folder, 'POE Documents')

In [36]:
#move all poe documents into documents folder
move_file('19oj4tcbwqgRsGEAia15LRAyVKROwUE_F', documents_folder)

True

In [17]:
#move all loose documents from evidence folder to documents folder
exclusion_array = ['documents', 'exams', 'Post Assessment Documents', 'technical reports', 'video demonstrations', 'video presentations']
move_file('16dCBkcvMqP5xURfMNBSRZIb98jIyPNrP', documents_folder, exclusion_array)


True

In [18]:
#move all personal documents into documents folder
evidence_folders = list_files('16dCBkcvMqP5xURfMNBSRZIb98jIyPNrP')
for folder in evidence_folders:
    if folder['title'] == 'documents':
        move_file(folder['id'], documents_folder)

**Presentation Folder**

In [19]:
#create presentation folder
presentation_folder = create_folder(poe_folder, 'Module 1 - Presentation')

In [20]:
#add presentation instruments folder
presentation_instruments_folder = create_folder(presentation_folder, 'instruments')

In [21]:
#add presentation evidence folder
presentation_evidence_folder = create_folder(presentation_folder, 'evidence')

In [22]:
#move all presentation evidence into presentation evidence folder
for folder in evidence_folders:
    if folder['title'] == 'video presentations':
        move_file(folder['id'], presentation_evidence_folder)

In [ ]:
#move all presentation post assessments docs into presentation folder
exclusion_array = ['demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
for folder in evidence_folders:
    if folder['title'] == 'Post Assessment Documents':
        move_file(folder['id'], presentation_folder,exclusion_array)
        

**Demonstration Folder**

In [23]:
#create demonstration folder
demonstration_folder = create_folder(poe_folder, 'Module 2 - Demonstration')

In [24]:
#add demonstration instruments folder
demonstration_instruments_folder = create_folder(demonstration_folder, 'instruments')

In [25]:
#add demonstration evidence folder
demonstration_evidence_folder = create_folder(demonstration_folder, 'evidence')

In [26]:
#move all demonstration evidence into demonstration evidence folder
for folder in evidence_folders:
    if folder['title'] == 'video demonstrations':
        move_file(folder['id'], demonstration_evidence_folder)

In [ ]:
#move all demonstration post assessments docs into demonstration folder
exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','exam_feedback.pdf','exam_review.pdf','report_feedback.pdf','report_review.pdf']
for folder in evidence_folders:
    if folder['title'] == 'Post Assessment Documents':
        move_file(folder['id'], demonstration_folder,exclusion_array)


**Reports Folder**

In [27]:
#create reports folder
reports_folder = create_folder(poe_folder, 'Module 3 - Reports')

In [28]:
#add reports instruments folder
reports_instruments_folder = create_folder(reports_folder, 'instruments')

In [29]:
#add reports evidence folder
reports_evidence_folder = create_folder(reports_folder, 'evidence')

In [30]:
#move all report evidence into report evidence folder
for folder in evidence_folders:
    if folder['title'] == 'technical reports':
        move_file(folder['id'], reports_evidence_folder)

In [ ]:
#move all reports post assessments docs into reports folder
exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','exam_feedback.pdf','exam_review.pdf']
for folder in evidence_folders:
    if folder['title'] == 'Post Assessment Documents':
        move_file(folder['id'], reports_folder,exclusion_array)


**Create Exams Folder**

In [31]:
#create exams folder
exams_folder = create_folder(poe_folder, 'Module 4 - Exams')

In [32]:
#add exams formative folder
exams_formatives_folder = create_folder(exams_folder, 'formatives')

In [33]:
#add exams summative folder
exams_summatives_folder = create_folder(exams_folder, 'summatives')

In [35]:
#move all exam evidence into exam evidence folder
for folder in evidence_folders:
    if folder['title'] == 'exams':
        exams_folders = list_files(folder['id'])
        for exam_folder in exams_folders:
            if exam_folder['title'] == 'formative':
                move_file(exam_folder['id'], exams_formatives_folder)
            if exam_folder['title'] == 'summative':
                move_file(exam_folder['id'], exams_summatives_folder)

In [ ]:
#move all exams post assessments docs into exams folder
exclusion_array = ['presentation_feedback.pdf','presentation_review.pdf','demonstration_feedback.pdf','demonstration_review.pdf','report_feedback.pdf','report_review.pdf']
for folder in evidence_folders:
    if folder['title'] == 'Post Assessment Documents':
        move_file(folder['id'], exams_folder,exclusion_array)


**Post Assessment Docs**

In [15]:
test = list_files('19oj4tcbwqgRsGEAia15LRAyVKROwUE_F')